# Llama 3를 KorQuad 데이터셋에 맞게 fine-tuning 하기

## 작성자 : AISchool ( http://aischool.ai/%ec%98%a8%eb%9d%bc%ec%9d%b8-%ea%b0%95%ec%9d%98-%ec%b9%b4%ed%85%8c%ea%b3%a0%eb%a6%ac/ )

In [ ]:
!nvidia-smi

## AutoTrain Advanced
https://github.com/huggingface/autotrain-advanced

In [ ]:
!pip install -q autotrain-advanced

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.5/277.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.4/150.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.6/174.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 33.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 16.

## PyTorch 업데이트

In [ ]:
!autotrain setup --update-torch

INFO     | 2024-05-08 14:35:10 | autotrain.cli.run_setup:run:43 - Installing latest xformers
INFO     | 2024-05-08 14:35:11 | autotrain.cli.run_setup:run:45 - Successfully installed latest xformers
INFO     | 2024-05-08 14:35:11 | autotrain.cli.run_setup:run:51 - Installing latest PyTorch
INFO     | 2024-05-08 14:35:17 | autotrain.cli.run_setup:run:53 - Successfully installed latest PyTorch


# HuggingFace Access Token 설정

In [ ]:
import os
os.environ['HF_TOKEN']="여러분의 HuggingFace Access Token값"

## Korquad 데이터셋에 맞게 Llama3 Fine-Tuning

- autotrain 설정값: https://github.com/huggingface/autotrain-advanced/blob/f1367b590dfc53d240e9684779991da540590386/src/autotrain/cli/run_llm.py#L21 (**과거 버전[0.6.35]**)
- autotrain 설정값: https://github.com/huggingface/autotrain-advanced/blob/main/src/autotrain/cli/run_llm.py#L17 (**최근 버전[0.6.80]**)


In [ ]:
# Data Augmentation 적용 o
!autotrain llm --train \
    --project-name "llama3-korquad-finetuning-da-8B" \
    --model "meta-llama/Meta-Llama-3-8B" \
    --data-path "korquad_prompt_da" \
    --text-column "text" \
    --peft \
    --quantization "int4" \
    --lr 2e-4 \
    --batch-size 8 \
    --epochs 40 \
    --trainer sft \
    --model_max_length 256

INFO     | 2024-05-08 14:35:41 | autotrain.cli.run_llm:run:343 - Running LLM
WARNING  | 2024-05-08 14:35:41 | autotrain.trainers.common:__init__:180 - Parameters supplied but not used: backend, func, deploy, inference, version, train, config
INFO     | 2024-05-08 14:35:41 | autotrain.backends.local:create:8 - Starting local training...
INFO     | 2024-05-08 14:35:41 | autotrain.commands:launch_command:327 - ['accelerate', 'launch', '--num_machines', '1', '--num_processes', '1', '--mixed_precision', 'no', '-m', 'autotrain.trainers.clm', '--training_config', 'llama3-korquad-finetuning-da-8B/training_params.json']
INFO     | 2024-05-08 14:35:41 | autotrain.commands:launch_command:328 - {'model': 'meta-llama/Meta-Llama-3-8B', 'project_name': 'llama3-korquad-finetuning-da-8B', 'data_path': 'korquad_prompt_da', 'train_split': 'train', 'valid_split': None, 'add_eos_token': False, 'block_size': -1, 'model_max_length': 256, 'padding': None, 'trainer': 'sft', 'use_flash_attention_2': False, 'log

# 학습결과 zip 파일로 압축후 다운로드

In [ ]:
import zipfile
import shutil
from google.colab import files

# 압축할 폴더 이름
folder_name = "llama3-korquad-finetuning-da-8B"  # Data Augmentation 적용 o

# 생성될 ZIP 파일 이름
zip_file_name = "llama3-korquad-finetuning-da-8B.zip" # Data Augmentation 적용 o

# 폴더를 ZIP 파일로 압축
shutil.make_archive(zip_file_name[:-4], 'zip', folder_name)

# ZIP 파일을 로컬로 다운로드
files.download(zip_file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>